# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [366]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize


## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [367]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [368]:
## your code to subset to one press release and take the string

subset = doj[doj['id'] == '17-1204']['contents']

pharma = subset.str.cat(sep=' ')

## proving conversion from panda series to a string 
type(pharma)


str

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [369]:
## A: your code here to restrict to alpha

def preprocess_text(text):

    punctuation = set(string.punctuation)
    
    words = text.split()
    
    cleaned_words = [''.join(char for char in word if char.isalpha()) for word in words]
    
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

preprocess_text(pharma)


'The founder and majority owner of Insys Therapeutics Inc was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain More than  Americans died of synthetic opioid overdoses last year and millions are addicted to opioids And yet some medical professionals would rather take advantage of the addicts than try to help them said Attorney General Jeff Sessions This Justice Department will not tolerate this We will hold accountable anyone  from street dealers to corporate executives  who illegally contributes to this nationwide epidemic And under the leadership of President Trump we are fully committed to defeating this threat to the American peopleJohn N Kapoor  of Phoenix Ariz a current member of the Board of Directors of Insys was arrested this morning in Arizona and charged with RICO conspiracy as well as other felonies including con

In [370]:
## B: your code here for part of speech tagging

tokens = word_tokenize(pharma) 

tokens_pos = pos_tag(tokens) 

In [371]:
## C: your code here to extract adjectives 

adjectives = [word for word, pos in tokens_pos if pos.startswith('JJ')]

adj_counts = {}

for adj in adjectives:
    if adj in adj_counts:
        adj_counts[adj] += 1
    else:
        adj_counts[adj] = 1
print(adj_counts)

sorted_adj_counts = sorted(adjectives.items(), key = lambda item: item[1], reverse=True)

# sort from most to fewest occurrences 
most_common_adjectives = sorted_adj_counts[:5]
most_common_adjectives

df_adjectives = pd.DataFrame(sorted_adj_counts, columns=['adjective', 'count'])
print(df_adjectives)


{'nationwide': 4, 'illegal': 2, 'More': 1, 'synthetic': 1, 'last': 1, 'medical': 1, 'accountable': 2, 'corporate': 2, 'American': 1, 'current': 2, 'other': 3, 'former': 8, 'federal': 2, 'later': 1, 'unsealed': 1, 'additional': 1, 'several': 1, 'various': 1, 'many': 1, 'fentanyl-based': 1, 'powerful': 2, 'narcotic': 1, 'intense': 1, 'large': 1, 'most': 1, 'reluctant': 1, 'non-cancer': 1, '“': 1, 'prior': 1, 'potent': 2, 'ongoing': 1, 'opioid': 5, 'street-level': 2, 'utilized': 1, 'acceptable': 1, 'addictive': 3, 'better': 1, 'important': 2, 'pharmaceutical': 1, 'responsible': 1, 'corrupt': 1, 'total': 1, 'patient': 2, 'epidemic': 1, 'dangerous': 1, 'Corporate': 1, 'aware': 1, 'public': 2, 'controlled': 1, 'special': 1, 'transparent': 1, 'regulatory': 1, 'followed.': 1, 'joint': 1, 'prosecute': 1, 'fraudulent': 1, 'Special': 1, 'military': 1, 'system.': 1, 'top': 1, 'private': 1, 'criminal': 1, 'proud': 1, 'high': 1, 'impact': 1, 'greater': 2, 'supervised': 2, 'pecuniary': 1, 'statutory'

AttributeError: 'list' object has no attribute 'items'

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [372]:
## your code here for part A

## resetting pharma 

subset_new = doj[doj['id'] == '17-1204']['contents']

pharma_new = subset.str.cat(sep=' ')

## using spacy 

spacy_pharma = nlp(pharma_new)

for one_tok in spacy_pharma.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DAT

In [373]:
## your code here for part B

law_ents = set(ent.text for ent in spacy_pharma.ents if ent.label_ == 'LAW')

print(law_ents)


{'the Controlled Substances Act', 'RICO'}


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [20]:
## C: summarizing RICO

# RICO, or the Racketeer Influenced and Corrupt Organizations, was a law passed in the 1970s primarily intended to target mafia cases, but 
# the flexibility of its statuatory language (engagement in a pattern of criminal activity connected to an enterprise) has allowed it to be 
# applied to cases involving drug cartels, corportations, and even politicians

# Link to Source Website: https://www.justia.com/criminal/docs/rico/

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [374]:
## D: extracting possible sentence lengths 

date_ents = [ent.text for ent in spacy_pharma.ents if ent.label_ == 'DATE' and re.search(r'\b(?:year|years)\b', ent.text, re.IGNORECASE)]

print(date_ents)

['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [375]:
## E: pull and print the original parts of the press releases where those year lengths are mentioned

pattern = r'([^.]*?years[^.]*\.)'

# adjust regex pattern so I'm matching to sentence! 

all_matchez = re.findall(pattern, pharma_new)

all_matchez

for match in all_matchez:
    print(match)


## Question: Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be 
## facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum).


['The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.',
 '\xa0 The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']

The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.


## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [376]:
## your code here for subsetting

doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])].reset_index()

doj_subset.shape

(717, 7)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [377]:

def clean_and_analyze_sentiment(text):

    tokenized = word_tokenize(text)
    tagged = pos_tag(tokenized)
    
    nlp_res = nlp(text)
    without_ner = " ".join([ent.text for ent in nlp_res if not ent.ent_type_])

    sent = SentimentIntensityAnalyzer()
    
    # sent score
    sentiment_scores = sent.polarity_scores(without_ner)
    
    # dictionary
    return sentiment_scores


In [ ]:
## B: your code here 

subset_3 = doj_subset[doj_subset['id'] == '17-1235']['contents']

pharma_3 = subset_3.str.cat(sep=' ')

pharma_3

## testing this on a specific case

results = [clean_and_analyze_sentiment(pharma_3) for contents in doj_subset]

results

## applying this to doj_subset 

doj_subset["sentiment_scores"] = doj_subset["contents"].apply(clean_and_analyze_sentiment)
print(doj_subset)

'A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.\xa0  \xa0 James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. \xa0 Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. \xa0 “Every citizen has the right to due process and protection from unreasonable force, and

[{'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992},
 {'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992}]

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [ ]:
## C: working with the four sentiment scores 

doj_subset_wscore = doj_subset.copy().reset_index()

sentiment_scores = doj_subset_wscore['contents'].apply(clean_and_analyze_sentiment)
doj_subset_wscore['neg_score'] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x['neg'])
doj_subset_wscore['pos_score'] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x['pos'])
doj_subset_wscore['neu_score'] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x['neu'])
doj_subset_wscore['compound_score'] = sdoj_subset_wscore["sentiment_scores"].apply(lambda x: x['compound'])

## sorting from highest to lowest

doj_subset_wscore = doj_subset_wscore.sort_values(by = 'neg', ascending = False)

# print of most negative press releases
top_neg = doj_subset_wscore.head(2)

top_neg[['id', 'contents', 'neg']]

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [ ]:
## D: agg and find the mean compound score by topic

print("Mean Compound Sentiment Score by Topic", doj_subset_wscore.groupby("topics_clean")['compound_score'].agg('mean'))

In [ ]:
## E: interpreting results 

# We see variation in scores because of the connotations of the three categories. There is an overwhelmingly positive connotation associated with
# civil rights, so it makes sense the the sentiment score is positive. There is a strong negative connotation associated with hate crimes, thus
# it makes sense that the sentiment scores for hate crimes is very close to -1. Project Safe Childhood is less obviously positive or negative by 
# looking at the title of the project alone, so a slightly negative sentiment score would make sense. 


## Link for Project Safe Childhood: https://www.justice.gov/psc


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [ ]:
subset_4 = doj_subset_wscore[doj_subset_wscore['id'] == '16-718']['contents']

pharma_4 = subset_4.str.cat(sep=' ')

pharma_4

In [ ]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]


In [ ]:
## A: writing a function

porter = SnowballStemmer(language = "english")

## convert to lowercase and a list

corpus_lower = doj_subset_wscore.contents.str.lower().to_list()

## use wordpunct tokenize and filter out with one

def process_step1(text):   
    
    stop_words = set(stopwords.words('english')).union(custom_doj_stopwords)

    nostop_listing1 = [word for word in wordpunct_tokenize(text)
                      if word not in stop_words]
    
    processed = [porter.stem(word) for word in nostop_listing1
                    if word.isalpha() 
                    and len(word) >= 4]
    
    processed_str = " ".join(processed)
    
    return(processed_str)

results_2 = [process_step1(text) for text in doj_subset_wscore['contents']]

results_2


In [ ]:
## B: applying and creating a new column 

doj_subset_wscore['processed_text'] = [process_step1(content) for content in doj_subset_wscore['contents']]

doj_subset_wscore

In [ ]:
## C: printing id, contents, and processed_text for examples 

row_16718 = doj_subset_wscore[doj_subset_wscore["id"] == "16-718"]
row_16217 = doj_subset_wscore[doj_subset_wscore["id"] == "16-217"]

print(row_16718[['id', 'contents', 'processed_text']])
print(row_16217[['id', 'contents', 'processed_text']])



## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [287]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [ ]:
# A: create_dtm 

dtm_nopre = create_dtm(list_of_strings = doj_subset_wscore["processed_text"],
                metadata = doj_subset_wscore[['id', 'compound_score', 'topics_clean']])

dtm_nopre


In [ ]:
## B: print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

## creating get_topwords

def get_topwords(dtm, num_words = 10):
    
    word_counts = dtm_nopre[dtm.columns[4:]].sum(axis=0).sort_values(ascending = False)
    
    return word_counts.head(num_words)

# threshold for top 5% of scores
top_5 = dtm_nopre['compound_score'].quantile(0.95)

# subsets of compounds
five_percent = dtm_nopre[dtm_nopre['compound_score'] >= top_5]

# print statements 
print("Top 10 words for press releases with compount sentiment in the top 5%", get_topwords(five_percent))


In [313]:
## C: print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

low_5 = dtm_nopre['compound'].quantile(0.05)

five_below = dtm_nopre[dtm_nopre['compound'] <= low_5]

five_below = five_below.drop_duplicates()

word_counts_2 = five_below.drop(columns = ['id', 'compound', 'topics_clean']).sum()

low_10 = word_counts_2.sort_values(ascending = False).head(10)

print("Top 10 words for press releases with compound sentiment in the top 5%", low_10)

ValueError: cannot reindex on an axis with duplicate labels

In [314]:
## D: Print the top 10 words for press releases in each of the three `topics_clean`

topics = ['Hate Crimes', 'Civil Rights', 'Project Safe Childhood']

concat_press = {}

for topic in topics:
    press_topic = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]['contents'].tolist()

    concat_press[topic] = ' '.join(press_topic)

for topic in topics:    
    dtm_press = create_dtm(list_of_strings = concat_press,
                            metadata = doj_subset_wscore[['id', 'topics_clean']])

    word_counts = {}
    for press_release_dtm in dtm_press:
        for word in press_release_dtm:
            word_counts[word] = word_counts.get(word, 0) + 1
    
    # Sort words by frequency and print the top 10 words
    sorted_word_counts = sorted(word_counts.items(), key = lambda x: x[1], reverse = True)
    for word, count in sorted_word_counts[:10]:
        print(f"{word}: {count}")
    print()



i: 8
e: 6
c: 6
d: 4
t: 4
o: 4
s: 4
h: 4
l: 3
a: 3

i: 8
e: 6
c: 6
d: 4
t: 4
o: 4
s: 4
h: 4
l: 3
a: 3

i: 8
e: 6
c: 6
d: 4
t: 4
o: 4
s: 4
h: 4
l: 3
a: 3



## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [49]:
## Step 1: re-tokenize and store in list
## here, i'm doing with the raw random sample of text
## in activity, you should do with the preprocessed texts

text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  doj_subset_wscore.contents]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### eliminated from the listing bc they're not in filtered dictionary.
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True, 
                                         random_state = 91988)


{0: ',', 1: '-', 2: '.', 3: '.,', 4: '10'}

Filtering out very rare and very common words reduced the length of dictionary from 12590 to 903.


{0: ',', 1: '-', 2: '.', 3: '.,', 4: '10'}

Sample of documents represented in dictionary format (with omitted words noted):


[([(0, 17),
   (1, 1),
   (2, 16),
   (3, 1),
   (4, 1),
   (5, 1),
   (6, 1),
   (7, 1),
   (8, 1),
   (9, 1),
   (10, 1),
   (11, 2),
   (12, 1),
   (13, 1),
   (14, 1),
   (15, 1),
   (16, 1),
   (17, 2),
   (18, 2),
   (19, 2),
   (20, 1),
   (21, 1),
   (22, 1),
   (23, 1),
   (24, 2),
   (25, 5),
   (26, 1),
   (27, 1),
   (28, 2),
   (29, 1),
   (30, 1),
   (31, 1),
   (32, 3),
   (33, 1),
   (34, 1),
   (35, 1),
   (36, 6),
   (37, 1),
   (38, 1),
   (39, 2),
   (40, 2),
   (41, 1),
   (42, 1),
   (43, 2),
   (44, 15),
   (45, 1),
   (46, 1),
   (47, 1),
   (48, 2),
   (49, 1),
   (50, 2),
   (51, 1),
   (52, 6),
   (53, 3),
   (54, 1),
   (55, 1),
   (56, 2),
   (57, 1),
   (58, 1),
   (59, 2),
   (60, 1),
   (61, 1),
   (62, 1),
   (63, 1),
   (64, 1),
   (65, 3),
   (66, 1),
   (67, 1),
   (68, 1),
   (69, 1),
   (70, 1),
   (71, 1),
   (72, 1),
   (73, 5),
   (74, 1),
   (75, 5),
   (76, 1),
   (77, 1),
   (78, 1),
   (79, 1),
   (80, 3),
   (81, 1),
   (82, 1),
   (83, 1),

In [50]:
## B: print the top 15 words in each topic 

topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)
    print()
    

(0, '0.071*"." + 0.060*"," + 0.052*"of" + 0.051*"the" + 0.040*"and" + 0.030*"to" + 0.021*"in" + 0.016*"a" + 0.014*"’" + 0.012*"s" + 0.011*"S" + 0.010*"U" + 0.010*"Attorney" + 0.009*"by" + 0.009*"was"')

(1, '0.063*"the" + 0.063*"." + 0.062*"," + 0.045*"of" + 0.036*"and" + 0.029*"to" + 0.022*"in" + 0.022*"a" + 0.011*"’" + 0.011*"s" + 0.010*"Attorney" + 0.010*"for" + 0.010*"The" + 0.010*"that" + 0.009*"by"')

(2, '0.068*"the" + 0.050*"." + 0.048*"," + 0.037*"of" + 0.035*"and" + 0.033*"to" + 0.019*"in" + 0.017*"a" + 0.016*"that" + 0.016*"’" + 0.016*"-" + 0.015*"The" + 0.015*"s" + 0.013*"with" + 0.011*"for"')



## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [162]:
## A: code to get doc-level topic probabilities 

corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### for each item in our original dictionary, get list of topic probabilities
l = [ldamod.get_document_topics(topic, minimum_probability = 0) for topic in corpus_fromdict]

### print result
l[0:5]

### write an assert statement 

assert len(l) == len(doj_subset_wscore), "Length of the list, l, and the dataframe, doj_subset_wscore are not equal"


[[(0, 0.00036581067), (1, 0.99931455), (2, 0.00031964845)],
 [(0, 0.0004575853), (1, 0.9991425), (2, 0.00039984199)],
 [(0, 0.0002844792), (1, 0.99946696), (2, 0.00024857657)],
 [(0, 0.00025112883), (1, 0.9995295), (2, 0.0002194352)],
 [(0, 0.00034774918), (1, 0.9993484), (2, 0.0003038668)]]

In [169]:
## B: add those topic probabilities to the dataframe

assert len(l) == len(doj_subset_wscore), "Length of l not equal to length of doj_subset_wscore"

document_topic_probabilities = []

for topic in corpus_fromdict:
    topic_probabilities = ldamod.get_document_topics(topic, minimum_probability = 0)

document_topic_probabilities.append([topic for _, topic in topic_probabilities])



In [52]:
## C: summarize the topic proportions for each of the topics_clean 

percent_top = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'], normalize='index') * 100

print(percent_top)


KeyError: 'top_topic'

In [86]:
## D: why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

example_1 = doj_subset_wscore[doj_subset_wscore['id'] == '14-248']

example_2 = doj_subset_wscore[doj_subset_wscore['id'] == '13-312']


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:

## A: creaing a new column — processed_texts_bigrams 

def create_bigram_onedoc(text):

    words = text.split()
    
    bigrams = ['_'.join(bigram) for bigram in zip(words, words[1:])]
    
    return " ".join(bigrams)

doj_subset_wscore['processed_text_bigrams'] = [create_bigram_onedoc(processed_content) for processed_content in doj_subset_wscore['processed_text']]


In [ ]:
## B: printing columns for press release with id = 16-217

rowz_16217 = doj_subset_wscore[doj_subset_wscore['id'] == '16-217']

rowz_16217[['id', 'processed_text', 'processed_text_bigrams']]



C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# C: creating dtm_bigram 

dtm_bigram = create_dtm(list_of_strings = doj_subset_wscore['processed_text_bigrams'],
                metadata = doj_subset_wscore[['id', 'topics_clean', 'compound_score']])



In [357]:
## D: printing the dimensions of dtm matrix and dtm_bigram matrix 

print(dtm_nopre.shape)

dtm_bigram.shape

## Comment on why the bigram matrix has more dimensions than the unigram matrix 

(717, 6905)


NameError: name 'dtm_bigram' is not defined

In [ ]:
## E: Find and print the 10 most prevelant bigrams for each of the three topics_clean using the get_topwords function from 2.2



# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 